## import necessary packages 

In [1]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

## read in CSVs and check first few rows 

In [2]:
calls_derecho = pd.read_csv('./data/911_Phone_Calls_Derecho_050320_050420.csv')
calls_tornado = pd.read_csv('./data/911_Phone_Calls_Tornado_030320.csv')
dispatch_derecho = pd.read_csv('./data/Computer_Aided_Dispatch_Data_Derecho_Incidents_050320-050420.csv', skiprows = 3)
dispatch_tornado = pd.read_csv('./data/Computer_Aided_Dispatch_Data_Tornado_Incidents_030320.csv', skiprows = 3)
zipcodes = gpd.read_file('./data/zipcodes.geojson')

## Rename columns 

In [3]:
calls_derecho = calls_derecho.rename(columns = {'Seizure DateTime': 'DateTime', 'CallTypeId': 'Call Type', 'ALI Latitude': 'Latitude', 'ALI Longitude': 'Longitude', 'Cell Tower Address': 'Cell Tower'})

In [4]:
calls_tornado = calls_tornado.rename(columns = {'Seizure DateTime': 'DateTime', 'CallTypeId': 'Call Type', 'ALI Latitude': 'Latitude', 'ALI Longitude': 'Longitude', 'Cell Tower Address': 'Cell Tower'})

In [5]:
dispatch_derecho = dispatch_derecho.rename(columns = {'IncidentDate': 'DateTime', 'Latitude1': 'Latitude', 'Longitude1': 'Longitude', 'IncidentTypeDescription1': 'Incident Description'})

In [6]:
dispatch_tornado = dispatch_tornado.rename(columns = {'IncidentDate': 'DateTime', 'Latitude1': 'Latitude', 'Longitude1': 'Longitude', 'IncidentTypeDescription1': 'Incident Description'})

## Add geometry points to data sets 

In [7]:
calls_derecho['Geometry'] = calls_derecho.apply(lambda x: Point((float(x.Longitude), 
                                                         float(x.Latitude))), 
                                        axis=1)

In [8]:
calls_tornado['Geometry'] = calls_tornado.apply(lambda x: Point((float(x.Longitude), 
                                                         float(x.Latitude))), 
                                        axis=1)

In [9]:
dispatch_derecho['Geometry'] = dispatch_derecho.apply(lambda x: Point((float(x.Longitude), 
                                                         float(x.Latitude))), 
                                        axis=1)

In [10]:
dispatch_tornado['Geometry'] = dispatch_tornado.apply(lambda x: Point((float(x.Longitude), 
                                                         float(x.Latitude))), 
                                        axis=1)

## check data

In [11]:
calls_derecho.head()

,DateTime,Call Type,Latitude,Longitude,Cell Tower,Geometry
0,5/3/2020 16:30,911 Calls,36.247802,-86.716847,WIRELESS CALLER,POINT (-86.716847 36.247802)
1,5/3/2020 16:30,911 Calls,36.143108,-86.800621,1161 21ST AV S - SE,POINT (-86.80062099999999 36.14310800000001)
2,5/3/2020 16:30,911 Calls,36.100516,-87.056329,8013 C MCCRORY LN - N,POINT (-87.05632900000001 36.100516)
3,5/3/2020 16:30,911 Calls,36.130021,-86.927819,5758 RIVER RD - SW,POINT (-86.927819 36.130021)
4,5/3/2020 16:31,911 Calls,36.142380,-86.881882,3744B ANNEX AVE - SE,POINT (-86.88188199999999 36.14238)


In [12]:
calls_tornado.head()

,DateTime,Call Type,Latitude,Longitude,Cell Tower,Geometry
0,3/3/2020 0:34,911 Calls,36.072708,-86.665779,620 Richards RD - N Sector,POINT (-86.665779 36.072708)
1,3/3/2020 0:36,911 Calls,36.159634,-86.841904,4507D KENTUCKY AV - SE,POINT (-86.841904 36.159634)
2,3/3/2020 0:36,911 Calls,36.264153,-86.708862,329 THELMA ST - S Sector,POINT (-86.708862 36.264153)
3,3/3/2020 0:36,911 Calls,NaN,NaN,526 FOUNDRY DR,POINT (nan nan)
4,3/3/2020 0:36,911 Calls,36.233467,-86.721077,5007B RUSKIN AVE - N,POINT (-86.72107700000001 36.233467)


In [13]:
dispatch_derecho.head()

,DateTime,Location,Latitude,Longitude,Incident Description,Geometry
0,5/3/2020 4:31:07 PM,901 KENWICK CT W,36.046877,-86.962526,ELECTRICAL HAZARD,POINT (-86.96252641 36.04687715)
1,5/3/2020 4:32:18 PM,8036 ARBOR DR,36.082491,-86.940680,ELECTRICAL HAZARD,POINT (-86.94067952 36.08249081)
2,5/3/2020 4:32:23 PM,119 DUE WEST AV E,36.248029,-86.716875,NOT ALERT COPD,POINT (-86.71687484 36.24802948)
3,5/3/2020 4:33:04 PM,630 GALLATIN PKE S,36.256491,-86.714817,ABDOMINAL PAIN,POINT (-86.71481656 36.25649117)
4,5/3/2020 4:33:52 PM,615 W HILLWOOD DR,36.129690,-86.879834,ELECTRICAL HAZARD,POINT (-86.87983392 36.12968971)


In [14]:
dispatch_tornado.head()

,DateTime,Location,Latitude,Longitude,Incident Description,Geometry
0,3/3/2020 12:36:34 AM,6205 COCKRILL BEND CIR,36.182002,-86.894052,GENERAL ALARM COMMERCIAL INDUSTRIAL BUILD,POINT (-86.89405212 36.18200191)
1,3/3/2020 12:37:24 AM,7337 COCKRILL BEND BLVD,36.175666,-86.894978,GENERAL ALARM COMMERCIAL INDUSTRIAL BUILD,POINT (-86.89497756999999 36.175666)
2,3/3/2020 12:38:17 AM,110 TUNE AIRPORT DR,36.179299,-86.883727,STRUCTURE FIRE COMMERCIAL INDUSTRIAL,POINT (-86.8837274 36.17929946)
3,3/3/2020 12:39:40 AM,555 MARRIOTT DR,36.151108,-86.689921,GENERAL ALARM COMMERCIAL INDUSTRIAL BUILD,POINT (-86.6899214 36.1511077)
4,3/3/2020 12:39:42 AM,1414 COUNTY HOSPITAL RD,36.177926,-86.845934,GENERAL ALARM COMMERCIAL INDUSTRIAL BUILD,POINT (-86.84593411 36.17792604)


In [15]:
zipcodes.head()

,zip,objectid,po_name,shape_stlength,shape_starea,geometry
0,37115,1,MADISON,178783.02488886821,596553400.57885742,"MULTIPOLYGON (((-86.68725 36.31821, -86.68722 ..."
1,37216,3,NASHVILLE,75820.997821400058,188884682.28344727,"MULTIPOLYGON (((-86.73451 36.23774, -86.73425 ..."
2,37204,9,NASHVILLE,93180.292250425613,200664795.51708984,"MULTIPOLYGON (((-86.77914 36.13424, -86.77923 ..."
3,37027,11,BRENTWOOD,159760.6942933173,174978422.04101562,"MULTIPOLYGON (((-86.81258 36.06319, -86.81263 ..."
4,37064,18,FRANKLIN,28995.828320601937,46969608.005737305,"MULTIPOLYGON (((-87.02197 36.01200, -87.02140 ..."


## shape check

In [16]:
calls_derecho.shape

(1549, 6)

In [17]:
calls_tornado.shape

(475, 6)

In [18]:
dispatch_derecho.shape

(627, 6)

In [19]:
dispatch_tornado.shape

(260, 6)

## count Cell Towers and Incident Descriptions

In [20]:
calls_derecho['Cell Tower'].value_counts()

WIRELESS CALLER                     339
4400 HARDING PIKE - E                15
3901 GRANNY WHITE PIKE - SE          14
WIRELESS 911 CALLER                  14
3009D DARRINGTON WAY - N             14
                                   ... 
1548 HUDSON RD - N                    1
751 S. 5TH ST. - E SECTOR             1
2404 OAKLAND AVE                      1
4940 OLD HICKORY BLVD - N SECTOR      1
324 RURAL HILL RD - N SECTOR          1
Name: Cell Tower, Length: 630, dtype: int64

In [21]:
calls_tornado['Cell Tower'].value_counts()

WIRELESS CALLER                       101
907 12TH AV N - N                      16
1051G CHICAMAUGA AVE - S               10
3333 LEBANON PIKE - SE                  8
4940 OLD HICKORY BLVD. - SE SECTOR      7
                                     ... 
121 4TH AV - SECTOR N S                 1
3811 CROUCH DR                          1
1415 FATHERLAND ST                      1
6292 FRUDEMAN RD - SE                   1
2955 BRICK CHURCH PKWY - N SECTOR       1
Name: Cell Tower, Length: 211, dtype: int64

In [22]:
dispatch_derecho['Incident Description'].value_counts()

ELECTRICAL HAZARD                              370
GENERAL ALARM COMMERCIAL   INDUSTRIAL BUILD     23
MVA INJURIES:                                    8
CARBON MONOXIDE ALARM HOUSE                      6
UNKNOWN STATUS:                                  6
                                              ... 
WATERFLOW ALARM APARTMENT                        1
CHEST PAIN - BREATHING NORMALLY <35 YEARS        1
OVERDOSE UNCONSCIOUS - ACCIDENTAL                1
OVERDOSE - ACCIDENTAL NO PRIORITY SYMPTOMS       1
HIGH RISK COMPLICATIONS                          1
Name: Incident Description, Length: 122, dtype: int64

In [23]:
dispatch_tornado['Incident Description'].value_counts()

GENERAL ALARM COMMERCIAL   INDUSTRIAL BUILD    27
OUTSIDE ODOR                                   26
WIRES DOWN WITH NO SMOKE OR ARCING             17
SMALL BUILDING INVESTIGATION                   15
ELECTRICAL HAZARD WITH OR NEAR WATER           12
                                               ..
TRAUMATIC INJURIES NOT ALERT                    1
MEDICAL ALARM NOTIFICATION                      1
HIGH LIFE HAZARD   ODOR WITH PATIENT            1
HEART PROBLEMS UNKNOWN STATUS                   1
OTHER PAIN                                      1
Name: Incident Description, Length: 90, dtype: int64

## count null values

In [24]:
calls_derecho.isnull().sum()

DateTime       0
Call Type      0
Latitude      66
Longitude     66
Cell Tower    24
Geometry       0
dtype: int64

In [25]:
calls_tornado.isnull().sum()

DateTime       0
Call Type      0
Latitude      25
Longitude     25
Cell Tower     7
Geometry       0
dtype: int64

In [26]:
dispatch_derecho.isnull().sum()

DateTime                0
Location                0
Latitude                0
Longitude               0
Incident Description    0
Geometry                0
dtype: int64

In [27]:
dispatch_tornado.isnull().sum()

DateTime                0
Location                0
Latitude                0
Longitude               0
Incident Description    0
Geometry                0
dtype: int64

map1 = sns.FacetGrid(dispactch_derecho, col = 'Incident Description')
map1.map(sns.barplot, "DateTime")

plt.hist(dispatch_derecho['Incident Description'], bins = 10);

## nashville map 

In [28]:
nashville = (36.16784, -86.77816)

nash_map = folium.Map(location = nashville, zoom_start = 10)

In [29]:
nash_map

In [30]:
#poly = zipcodes.loc[zipcodes['zip'] == '37207']
#polygon37207.shape